In [ ]:
import pandas as pd
import numpy as np
import scipy as scipy
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from pylab import rcParams
from pandas import Series, DataFrame
from scipy import stats
from scipy.stats import f_oneway
from scipy.stats import ttest_ind
from statsmodels.stats.proportion import proportions_ztest
from statsmodels.stats.power import tt_ind_solve_power
import statsmodels.stats.api as sms
import xlsxwriter

### 1. Before running the test

#### Define sample size (check the R script for sensibility sample size and effect size). Highly recommend using Stata for it, but if not available R script will get close.

#### If a true random sample is not viable, challenge the test and control group in different dimensions (skip those three tests if you can create a true random sample):

#### 1.1 Z-test: two proportions test

In [ ]:
proportions_ztest(p1,p2, alternative='two-sided')

#### 1.2 Chi-test: Categorical Value Test

In [ ]:
stats.chi2_contingency(cat)

#### 1.3 t-test: Mean test

In [ ]:
ttest_ind(m1, m2)

#### 2. After the test is run. Test the results and draw inferences

#### 2.1 Define function for permutation test

In [ ]:
def perm_func(x, nA, nB):
   n = nA + nB
   id_B = set(random.sample(range(n), nB))
   id_A = set(range(n)) - id_B
   return x.loc[id_B].mean() - x.loc[id_A].mean()

####  2.2. Calculate the observed test x control difference

In [ ]:
control = (df['Control0xTest1']==0) 
test = (df['Control0xTest1']==1) 
obs = 100*(np.array(df[control]['rate']) - np.array(df[test]['rate']))
print(f'Observed difference: {obs[0]:.4f}%')

#### where x1 and x2 being each two steps that make the rate in the previous step
#### Create arrays to do the permutation test (p1 array with 0, p2 array with 1), then join the arrays in a pd as p

In [ ]:
dif = (np.array(df[control]['x1']) + np.array(df[test]['x1']) 
                 -np.array(df[control]['x2']) - np.array(df[test]['x2]))
dif[0]
p1 = ([0] * dif[0])
dif_w = (np.array(df[control]['x2']) + np.array(df[test]['x2']))
p2 = ([1] * dif_w[0])
p1.extend(p2)
p = pd.Series(p1)

#### Run as many permutations as desired (change range)

In [ ]:
perm_diffs = [100 * perm_func(p, np.array(df[control]['x1'])[0],np.array(df[test]['x1'])[0])
              for i in range(1000)]
perm_diffs

#### Calculate p-value for permutation test

In [ ]:
if obs < 0:
    print(np.mean([diff < obs for diff in perm_diffs]))
else:
    print(np.mean([diff > obs for diff in perm_diffs]))

#### p-value approximation for a binomial distribution (will be different than permutation test)

In [ ]:
chi = np.array([[np.array(df[control]['x2']), 
                      np.array(df[control]['x1'])-np.array(df[control]['x2'])],
                      [np.array(df[test]['x2']), 
                      np.array(df[test]['x1'])-np.array(df[test]['x2'])
                      ]])
chi2, p_value, df, _ = stats.chi2_contingency(bm)
print(f'p=value for single sided test: {p_value/2:4f}')

#### Plot

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
ax.hist(perm_diffs, rwidth=0.9)
ax.axvline(x=obs_bm, lw=2)
ax.text(-.18, 200, 'Observed\ndifference', bbox={'facecolor':'white'})
ax.set_xlabel('Rate (in percentage)')
ax.set_ylabel('Frequency')
plt.show()